## Import

In [ ]:
import sys
sys.dont_write_bytecode = True

from functions.map import *
from functions.rl import *

## Map tests

In [ ]:
m = map()
map_filename = "0"
m.load("../maps/" + map_filename + ".map")
# m.draw(filename = "../maps/" + map_filename + ".png",
# 	figure_size = 15)

In [ ]:
m = map()
map_filename = "1"
m.load("../maps/" + map_filename + ".map")
# m.draw(filename = "../maps/" + map_filename + ".png",
# 	figure_size = 15)

In [ ]:
seed = random.randint(0, 1000)
random.seed(seed)
k = 3
m = map(generate = k)
map_filename = "gen_0"
# m.draw(filename = "../maps/" + map_filename + ".png",
# 	seed = seed,
# 	figure_size = 15)

# save
# m.save("../maps/" + map_filename + ".map")
# load from last save
# m = map()
# m.load("../maps/" + map_filename + ".map")
# fig = m.draw(figure_size = 15)

## Reinforcement tests

Proposed action for the state $s$ at step $k$, with $\mathcal{A}(s)$ set of available actions from the state $s$:
$$ a^*_{k}\left(s\right) = \argmax\limits_{a \, \in \, \mathcal{A}(s)} \texttt{action\_value}(s, a) $$

$\varepsilon$-greedy policy at step $k$:
$$ \pi_{k}\left(a \,|\, s\right) =
\begin{cases}
	1 - \varepsilon + \frac{\varepsilon}{\mathcal{A}(s)} & \text{if } a = a^*_{k}\left(s\right)  \\
	\frac{\varepsilon}{\mathcal{A}(s)} & \text{if } a \in \mathcal{A}(s) \setminus \{a^*_{k}\left(s\right)\}
\end{cases}
$$

In [ ]:
seed = random.randint(0, 1000)
# seed = 949 # good example k = 4
# seed = 801 # good example k = 4
# seed = 59 # good example k = 4
# seed = 921 # good example k = 5
# seed = 230 # fantastic k = 5
# seed = 
random.seed(seed)
k = 5
m = map(generate = k)
m.draw(filename = "../maps/" + map_filename + ".png",
    seed = seed,
	figure_size = 10)

for gamemode in ["shortest_path", "survivor", "explorer"]:
	out = False
	n_iteration = 0
	while not(out) and n_iteration < 100:
		out = on_policy_first_visit_mc_control(m, gamemode)
		n_iteration += 1
		# DEBUG
		print(str(n_iteration) + " ", end = "")
	if n_iteration < 100:
		stamina, life, path = out
		print(gamemode + " -> " + str(life) + ": (" + str(stamina) + ", " + str(path) + ")")	
	else:
		print(gamemode + " -> path not found")

# m._stamina_life

### TODO
- book: https://www.andrew.cmu.edu/course/10-703/textbook/BartoSutton.pdf chapter 5
- link reinforcement shortest path: https://medium.com/betacom/how-to-find-the-shortest-path-using-reinforcement-learning-4a1bb21c2fb5